In [1]:
from ortools.linear_solver import pywraplp
from ortools.linear_solver import pywraplp
import numpy as np
from itertools import combinations
import pandas as pd
import numpy as np

In [2]:
dose_matrix = pd.read_csv('DoseMatrix.csv')

In [3]:
dose_matrix =np.array(dose_matrix)

In [4]:
CTV_index=[167, 168, 169, 170, 171, 172, 173, 186, 187, 188, 189, 190, 
     191, 192, 193, 194, 206, 207, 208, 209, 210, 211, 212, 213, 
     214, 227, 228, 229, 230, 231, 232, 233]

bladder_index=[87, 88, 89, 90, 91, 92, 93, 106, 107, 108, 109, 110, 
         111, 112, 113, 114, 126, 127, 128, 129, 130, 131, 132, 
         133, 134, 147, 148, 149, 150, 151, 152, 153]

Rectal_index=[248, 249, 250, 251, 252, 267, 268, 269, 270, 271, 272, 
        273, 287, 288, 289, 290, 291, 292, 293, 308, 309, 310, 311, 312]

Un_index=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 
             21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 
             41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 
             61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 
             81, 82, 83, 84, 85, 86, 94, 95, 96, 97, 98, 99, 100, 101, 104, 105, 115, 118, 119, 120,
             125, 139, 140, 145, 146, 154, 159, 160, 165, 166, 174, 179, 180, 185, 199, 200, 205, 219, 220, 
             225, 226, 234, 239, 240, 241, 244, 245, 246, 247, 253, 254, 255, 258, 259, 260, 261, 262, 263, 
             264, 265, 266, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 294, 295, 296, 
             297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 313, 314, 315, 316, 317, 318, 319, 320, 
             321, 322, 323, 324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 337, 338, 339, 
             340, 341, 342, 343, 344, 345, 346, 347, 348, 349, 350, 351, 352, 353, 354, 355, 356, 357, 358, 
             359, 360, 361, 362, 363, 364, 365, 366, 367, 368, 369, 370, 371, 372, 373, 374, 375, 376, 377, 
             378, 379, 380, 381, 382, 383, 384, 385, 386, 387, 388, 389, 390, 391, 392, 393, 394, 395, 396, 
             397, 398, 399]

Left_index=[116, 117, 135, 136, 137, 138, 155, 156, 157, 158, 175, 176, 177, 178,
       195, 196, 197, 198, 215, 216, 217, 218, 235, 236, 237, 238, 256, 257]

Right_index=[101, 102, 120, 121, 122, 123, 140, 141, 142, 143, 160, 161, 162, 
       163, 180, 181, 182, 183, 200, 201, 202, 203, 220, 221, 222, 223, 241, 242]

In [5]:
solver = pywraplp.Solver('Final Project',
                         pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
objective=solver.Objective() 
objective.SetMinimization()

# Decision Variables
beamlets = []
for i in range(60):
    beamlets.append(solver.NumVar(0,solver.infinity(), 'beamlets[%d]' % i))
voxels = []    
for i in range(400):
    voxels.append(solver.NumVar(0,solver.infinity(),'voxels[%d]' % i))

    
#Constraints
voxel_constraints = [0 for _ in range(400)]
for i in range(400):
    voxel_constraints[i] = solver.Constraint(0,0)
    for j in range(60):
        voxel_constraints[i].SetCoefficient(beamlets[j],dose_matrix[i,j+1])
    voxel_constraints[i].SetCoefficient(voxels[i], -1)
        
######################################################## CTV#################################################

# every voxel receives a uniform dose of 82.8       ( = 82.8)
z_p = [0 for _ in range(len(CTV_index))]
z_n = [0 for _ in range(len(CTV_index))]

for i in range(len(CTV_index)):
    z_p[i] = solver.NumVar(0, 82.8*0.08, "Z plus CTV %d" %i)
    z_n[i] = solver.NumVar(0, 82.8*0.05, "Z minus CTV %d" %i)
     
CTV_constraint = [0 for _ in range(len(CTV_index))]
for i in range(len(CTV_index)):
    CTV_constraint[i] = solver.Constraint(82.8,82.8)
    
    for j in range(60):
        CTV_constraint[i].SetCoefficient(beamlets[j],dose_matrix[CTV_index[i],j+1])
    CTV_constraint[i].SetCoefficient(z_p[i],1)
    CTV_constraint[i].SetCoefficient(z_n[i],-1)

    
    
############################################ bladder ####################################################

# # max dose to a voxel 81 ( < = 81)
z_b_1 = [0 for _ in range(len(bladder_index))]

for i in range(len(bladder_index)):
    z_b_1[i] = solver.NumVar(0, solver.infinity(), "z bladder #1 %d" %i)
    

bladder_constraint_1 = [0 for _ in range(len(bladder_index))]
for i in range(len(bladder_index)):
    bladder_constraint_1[i] = solver.Constraint(-solver.infinity(),81)
    for j in range(60):
        bladder_constraint_1[i].SetCoefficient(beamlets[j],dose_matrix[bladder_index[i],j+1])
    bladder_constraint_1[i].SetCoefficient(z_b_1[i],-1)


# Average dose should be <= 50     ( <= 50)

z_b_2 = solver.NumVar(0, solver.infinity(), "z bladder #2 %d" %i)
bladder_constraint_2 = solver.Constraint(-solver.infinity(), 50)
bladder_constraint_2.SetCoefficient(z_b_2, -1)

for i in bladder_index:
    bladder_constraint_2.SetCoefficient(voxels[i],1/len(bladder_index))

        
    
# at most 10% of the bladder should receive a dose >65          ( >65)
y = [0 for _ in range(len(bladder_index))]
z_y = solver.IntVar(0,1, "z y constraint bladder")

for i in range(len(bladder_index)):
     y[i] = solver.IntVar(0,1,"y bladder %d" %i)

bladder_constraint_3 = [0 for _ in range(len(bladder_index))]
for i in range(len(bladder_index)):
    bladder_constraint_3[i] = solver.Constraint(-solver.infinity(),65)
    for j in range(60):
        bladder_constraint_3[i].SetCoefficient(beamlets[j],dose_matrix[bladder_index[i],j+1])
    bladder_constraint_3[i].SetCoefficient(y[i],-100)

bladder_constraint_3b = solver.Constraint(-solver.infinity(),0.1*len(bladder_index))
for j in range(len(bladder_index)):
     bladder_constraint_3b.SetCoefficient(y[j],1)
bladder_constraint_3b.SetCoefficient(z_y, -1)

        
        

        
# ####################### Rectal########################################

#max dose to a voxel: 79.2              ( < = 79.2)

z_r_1 = [0 for _ in range(len(Rectal_index))]

for i in range(len(Rectal_index)):
    z_r_1[i] = solver.NumVar(0, solver.infinity(), "z rectal #1 %d" %i)
    

Rectal_constraint_1 = [0 for _ in range(len(Rectal_index))]
for i in range(len(Rectal_index)):
    Rectal_constraint_1[i] = solver.Constraint(-solver.infinity(),79.2)
    for j in range(60):
        Rectal_constraint_1[i].SetCoefficient(beamlets[j],dose_matrix[Rectal_index[i],j+1])
    Rectal_constraint_1[i].SetCoefficient(z_r_1[i],-1)

    
    
# #average dose should be <= 40

z_r_2 = solver.NumVar(0, solver.infinity(), "z rectal #2 %d" %i)
Rectal_constraint_2 = solver.Constraint(-solver.infinity(), 40)
Rectal_constraint_2.SetCoefficient(z_r_2, -1.0)

for i in Rectal_index:
    Rectal_constraint_2.SetCoefficient(voxels[i],1/len(Rectal_index))



# ############################## Unspecified ############################

# max dose to a voxel: 72         ( < =72)

z_u_1 = [0 for _ in range(len(Un_index))]

for i in range(len(Un_index)):
    z_u_1[i] = solver.NumVar(0, solver.infinity(), "z Unspecified #1 %d" %i)
    

Un_constraint_1 = [0 for _ in range(len(Un_index))]
for i in range(len(Un_index)):
    Un_constraint_1[i] = solver.Constraint(-solver.infinity(),72)
    for j in range(60):
        Un_constraint_1[i].SetCoefficient(beamlets[j],dose_matrix[Un_index[i],j+1])
    Un_constraint_1[i].SetCoefficient(z_u_1[i],-1)


#############################Left femur head ##########################

#max dose to a voxel: 50            ( <= 50)
  
z_l_1 = [0 for _ in range(len(Left_index))]

for i in range(len(Left_index)):
    z_l_1[i] = solver.NumVar(0, solver.infinity(), "z left femur #1 %d" %i)
    

left_constraint_1 = [0 for _ in range(len(Left_index))]
for i in range(len(Left_index)):
    left_constraint_1[i] = solver.Constraint(-solver.infinity(),50)
    for j in range(60):
        left_constraint_1[i].SetCoefficient(beamlets[j],dose_matrix[Left_index[i],j+1])
    left_constraint_1[i].SetCoefficient(z_l_1[i],-1)    
        

        
# at most 10% of the left femur should receive a dose > 40 
y_left = [0 for _ in range(len(Left_index))]
z_y_left = solver.IntVar(0,1, "z y constraint left femur")

for i in range(len(Left_index)):
     y_left[i] = solver.IntVar(0,1,"y left femur %d" %i)

Left_constraint_3 = [0 for _ in range(len(Left_index))]
for i in range(len(Left_index)):
    Left_constraint_3[i] = solver.Constraint(-solver.infinity(),40)
    for j in range(60):
        Left_constraint_3[i].SetCoefficient(beamlets[j],dose_matrix[Left_index[i],j+1])
    Left_constraint_3[i].SetCoefficient(y_left[i],-100)

Left_constraint_3b = solver.Constraint(-solver.infinity(),0.15*len(Left_index))
for j in range(len(Left_index)):
     Left_constraint_3b.SetCoefficient(y_left[j],1)
Left_constraint_3b.SetCoefficient(z_y_left,-1)


############################# right femur head ##########################


#max dose to a voxel: 50               ( <= 50 )
z_rl_1 = [0 for _ in range(len(Right_index))]

for i in range(len(Right_index)):
    z_rl_1[i] = solver.NumVar(0, solver.infinity(), "z right femur #1 %d" %i)
    

Right_constraint_1 = [0 for _ in range(len(Right_index))]
for i in range(len(Right_index)):
    Right_constraint_1[i] = solver.Constraint(-solver.infinity(),50)
    for j in range(60):
        Right_constraint_1[i].SetCoefficient(beamlets[j],dose_matrix[Right_index[i],j+1])
    Right_constraint_1[i].SetCoefficient(z_rl_1[i],-1)    
        

        
# at most 10% of the right femur should receive a dose > 40 
y_right = [0 for _ in range(len(Right_index))]
z_y_right = solver.IntVar(0,2, "z y constraint right femur")

for i in range(len(Right_index)):
     y_right[i] = solver.IntVar(0,1,"y right femur %d" %i)

Right_constraint_3 = [0 for _ in range(len(Right_index))]
for i in range(len(Right_index)):
    
    # <= 40
    Right_constraint_3[i] = solver.Constraint(-solver.infinity(),40)
    for j in range(60):
        Right_constraint_3[i].SetCoefficient(beamlets[j],dose_matrix[Right_index[i],j+1])
    # Big M constraint
    Right_constraint_3[i].SetCoefficient(y_right[i],-100)

Right_constraint_3b = solver.Constraint(-solver.infinity(),0.15*len(Right_index))
for j in range(len(Right_index)):
     Right_constraint_3b.SetCoefficient(y_right[j],1)
Right_constraint_3b.SetCoefficient(z_y_right, -1)
        
        
########################## Objective Function ###########################

# CTV
for i in range(len(z_p)):
    objective.SetCoefficient(z_p[i],1000)
    objective.SetCoefficient(z_n[i],1000)

# Bladders
for i in range(len(z_b_1)):
    objective.SetCoefficient(z_b_1[i],1)

objective.SetCoefficient(z_b_2,100)

# Rectal Solids
for i in range(len(z_r_1)):
    objective.SetCoefficient(z_r_1[i],1)
    
objective.SetCoefficient(z_r_2,6000)

#Unspecified
for i in range(len(z_u_1)):
    objective.SetCoefficient(z_u_1[i],1)

# left femur head
for i in range(len(z_l_1)):
    objective.SetCoefficient(z_l_1[i],5000)

#right femur head
for i in range(len(z_rl_1)):
    objective.SetCoefficient(z_rl_1[i],1)

# bladder, left femur, right femur's at most constraint
objective.SetCoefficient(z_y,1)
objective.SetCoefficient(z_y_left,1)
objective.SetCoefficient(z_y_right,1)

status = solver.Solve()

if status == solver.OPTIMAL:
    print('Problem solved in %f milliseconds' %solver.wall_time())
elif status == solver.FEASIBLE:
    print('Solver claims feasibility but not optimality')
else:
    print('Solver ran to completion but did not find an optimal solution')

print('error',objective.Value()) 


#print(bladder_constraint_2.dual_value())

#print(solver.ExportModelAsLpFormat(False).replace('\\', '').replace(',_', ','), sep='\n')

Problem solved in 7296.000000 milliseconds
error 276454.3426513021


In [6]:
print(bladder_constraint_2.dual_value())

0.0


In [7]:
#print(solver.ExportModelAsLpFormat(False).replace('\\', '').replace(',_', ','), sep='\n')

## All Voxels

In [8]:
All_voxels = np.zeros((400)).astype(np.float)
for i in range(400):
    All_voxels[i] = voxels[i].solution_value()
#print(All_voxels.reshape(20,20))
All_voxels = pd.DataFrame(All_voxels.reshape((20,20)), index=[i for i in range(1,21)],
                          columns = [i for i in range(1,21)]).round(2)
All_voxels

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
1,68.34,55.43,46.17,44.71,46.78,43.26,30.57,15.82,6.60,3.35,3.33,5.58,10.53,16.20,19.06,19.21,19.46,20.47,20.64,19.22
2,68.78,63.03,52.77,47.95,50.24,50.03,39.53,23.85,11.89,6.13,4.94,7.54,14.00,21.47,25.49,25.60,24.55,23.26,21.22,19.08
3,57.36,63.37,59.65,54.06,53.53,53.56,45.89,32.18,19.64,11.83,9.17,11.94,19.79,28.63,32.83,31.49,27.76,23.94,21.07,19.59
4,38.72,52.88,59.97,60.54,59.45,56.76,49.02,38.09,28.16,20.95,17.70,20.16,28.28,37.03,39.70,35.34,28.65,23.82,21.64,20.28
5,21.03,35.82,50.44,61.10,65.83,62.40,51.80,41.24,35.32,32.36,30.96,32.98,39.44,45.59,44.59,36.74,28.70,24.49,22.41,19.11
6,9.20,19.64,34.91,52.72,66.98,68.79,57.37,44.98,41.30,43.83,46.80,49.25,52.42,53.32,47.51,37.48,29.70,25.40,21.13,15.05
7,3.25,8.83,20.22,38.95,60.24,71.14,65.00,53.08,49.23,54.34,61.25,65.00,65.00,60.59,51.11,40.00,31.23,24.14,16.70,9.50
8,0.95,3.43,10.54,26.25,49.33,67.78,71.08,65.00,61.65,65.00,71.18,75.02,74.52,69.15,58.44,44.57,31.13,19.69,10.70,4.75
9,0.28,1.45,6.11,18.95,41.05,62.90,73.99,76.18,76.18,76.18,76.31,77.52,80.01,79.22,68.62,49.19,29.04,14.30,5.87,1.97
10,0.21,1.25,5.67,18.26,40.31,62.75,76.18,82.80,86.21,84.34,78.66,76.96,82.82,86.80,76.18,52.24,27.83,11.77,4.04,1.13


In [9]:
OUTPUT  = 'output/cross_section.csv'
cross_section = All_voxels.to_csv(OUTPUT, index=[i for i in range(1,21)], columns = [i for i in range(1,21)])

## Check Constraint

**Beamlets Check**

In [10]:
# check beamlets value
beamlets_value = []
for i in range(60):
    beamlets_value.append(beamlets[i].solution_value())
print('Beamlets values are ')
print(np.array(beamlets_value))   

Beamlets values are 
[58.37244663  0.          0.          0.          0.          0.
  0.          0.         27.56820451 17.08266308  0.          0.
  0.          0.         11.70534165  0.          0.         27.73383687
 14.09110277  0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          1.74814961  0.
  0.          0.          0.          0.          0.          5.88528689
 23.02291508  0.          0.         54.16128161 16.51519369  0.
  0.          0.          0.          0.         34.17172399 11.95409216
 82.23993589  5.30475314  0.         90.58995754  0.          0.        ]


**CTV check**

In [11]:
#Every voxel receives a uniform dose of 82.8 
CTV_voxel = []
every = 0
for i in CTV_index:
    every = voxels[i].solution_value() 
    CTV_voxel.append(every)
print('CTV voxel dose(Gy)')
print(np.array(CTV_voxel))

CTV voxel dose(Gy)
[76.176      76.176      76.176      76.31239255 77.52465707 80.01477362
 79.21852355 76.176      82.8        86.20618394 84.34060445 78.66384695
 76.95978459 82.8150215  86.7987772  76.176      80.92812536 84.45751493
 86.48927114 84.94656625 80.2182888  78.93574129 84.08867192 86.94
 76.176      82.33561291 78.3050819  78.62346097 80.58763151 82.14991244
 82.8        79.88974016]


In [12]:
# the distance to 82.8
zp0 = np.array([[x.solution_value() for x in z_p]])
zn0 = np.array([[x.solution_value() for x in z_n]])
print('Positive distance to 82.8 \n',zp0)
print('\n')
print('Negative distance to 82.8 \n',zn0)

Positive distance to 82.8 
 [[6.624      6.624      6.624      6.48760745 5.27534293 2.78522638
  3.58147645 6.624      0.         0.         0.         4.13615305
  5.84021541 0.         0.         6.624      1.87187464 0.
  0.         0.         2.5817112  3.86425871 0.         0.
  6.624      0.46438709 4.4949181  4.17653903 2.21236849 0.65008756
  0.         2.91025984]]


Negative distance to 82.8 
 [[0.         0.         0.         0.         0.         0.
  0.         0.         0.         3.40618394 1.54060445 0.
  0.         0.0150215  3.9987772  0.         0.         1.65751493
  3.68927114 2.14656625 0.         0.         1.28867192 4.14
  0.         0.         0.         0.         0.         0.
  0.         0.        ]]


In [13]:
print('maximum difference of CTV voxels values to 82.8 are ', (max(CTV_voxel)-82.8)/82.8, (82.8- min(CTV_voxel))/82.8)

maximum difference of CTV voxels values to 82.8 are  0.05000000000000018 0.0800000000000008


In [14]:
print('maximum CTV voxels value is ', max(CTV_voxel))

maximum CTV voxels value is  86.94000000000001


In [15]:
print('minimum CTV voxels value is ', min(CTV_voxel))

minimum CTV voxels value is  76.17599999999993


In [16]:
print('percentage of voxels below 79:', sum((np.array(CTV_voxel) < 79)*1) / len(CTV_voxel))

percentage of voxels below 79: 0.40625


In [17]:
print('percentage of voxels Above 86.94:', sum((np.array(CTV_voxel) > 82.8*1.05)*1) / len(CTV_voxel))

percentage of voxels Above 86.94: 0.03125


**Bladder Check**

In [18]:
# Max dose to a voxel: 81
bladder_voxel = []
every = 0
for i in bladder_index:
    every = round(voxels[i].solution_value(),4)
    bladder_voxel.append(every)
print('Bladder Voxel Doses\n',np.array(bladder_voxel))

Bladder Voxel Doses
 [41.2362 35.3242 32.357  30.9568 32.9819 39.4401 45.5914 57.3682 44.9822
 41.3014 43.8335 46.803  49.247  52.4178 53.3218 47.5064 65.     53.0784
 49.2342 54.3434 61.2494 65.     65.     60.5866 51.1057 65.     61.6467
 65.     71.1772 75.0202 74.5153 69.1523]


In [19]:
# if any voxel has more than 81
zb1 = np.array([[x.solution_value() for x in z_b_1]])
print('distances to 81 if voxel value is above 81\n',zb1)

distances to 81 if voxel value is above 81
 [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0.]]


In [20]:
# check if there are voxel with value more than 81
print('are there values more than 81?')
print(max(bladder_voxel) > 81)

print('how many percentage of voxel values are more than 81?')

print(round(sum((np.array(bladder_voxel) >81)*1)/len(bladder_voxel),2)*100,'%')

are there values more than 81?
False
how many percentage of voxel values are more than 81?
0.0 %


In [21]:
# average dose shouldbe <= 50
print('average dose received by bladder voxels are ',sum(bladder_voxel)/len(bladder_voxel))
print('slack variable to bring down the value to below 50: ', z_b_2.solution_value())

average dose received by bladder voxels are  53.149321875000005
slack variable to bring down the value to below 50:  3.149317504407082


In [22]:
# at most 10% of the bladder should receive a dose > 65
y_value = np.array([t.solution_value() for t in y])
print('voxel that are larger that 65 are indicated by 1 \n',y_value)

voxel that are larger that 65 are indicated by 1 
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 1. 1. 1.]


In [23]:
test = np.array(list(map(lambda x: 1 if x > 65.00 else 0,bladder_voxel)))
print(round(sum(test)/len(test),4), ' of the voxels are above 65')

0.125  of the voxels are above 65


In [24]:
print('voxel that are larger that 65 are indicated by 1 \n',test)

voxel that are larger that 65 are indicated by 1 
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1]


In [25]:
y_value == test

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True])

In [26]:
print('the number of constraints we have to set loose to be above 65',z_y.solution_value())

the number of constraints we have to set loose to be above 65 1.0


**Rectal Solid Check**

In [27]:
# max dose to a voxel: 79.2
Rectal_voxel = []
every = 0
for i in Rectal_index:
    every = round(voxels[i].solution_value(),8)
    Rectal_voxel.append(every)
print('rectal voxel dose values')
print(np.array(Rectal_voxel))

rectal voxel dose values
[68.85636385 70.70545041 76.5937351  79.05186123 76.9087614  75.34816978
 64.37276419 63.83364172 65.23967131 65.44889946 66.13122495 64.87985567
 76.64018685 63.51276292 55.09860942 47.71345426 45.90807737 52.89409156
 60.30296609 59.31424662 41.85535663 29.45303549 28.61104644 40.81605524]


In [28]:
zr1 = np.array([[x.solution_value() for x in z_r_1]])
print('distances to 79.2 if voxel value is above 79.2\n',zr1)

distances to 79.2 if voxel value is above 79.2
 [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


In [29]:
# check if there's value larger than 79.2
print('are there values more than 79.2?')
print(max(Rectal_voxel) > 79.2)

print('how many percentage of voxel values are more than 79.2?')

print(round(sum((np.array(Rectal_voxel) >79.2)*1)/len(Rectal_voxel),2)*100,'%')

are there values more than 79.2?
False
how many percentage of voxel values are more than 79.2?
0.0 %


In [30]:
# average dose should be <= 40
print('average dose received by bladder voxels are ',sum(Rectal_voxel)/len(Rectal_voxel))
print('slack variable to bring down the value to below 40: ',z_r_2.solution_value())

average dose received by bladder voxels are  59.97876199833333
slack variable to bring down the value to below 40:  19.97876199913546


**Unspecified**

In [31]:
# Max dose to a voxel: 72
Un_voxel = []
every = 0
for i in Un_index:
    every = round(voxels[i].solution_value(),3)
    Un_voxel.append(every)
np.array(Un_voxel)

array([ 68.335,  55.429,  46.173,  44.714,  46.779,  43.255,  30.574,
        15.824,   6.602,   3.353,   3.331,   5.578,  10.526,  16.198,
        19.062,  19.206,  19.46 ,  20.474,  20.638,  19.218,  68.784,
        63.03 ,  52.765,  47.949,  50.241,  50.03 ,  39.531,  23.846,
        11.887,   6.131,   4.938,   7.543,  13.998,  21.475,  25.487,
        25.603,  24.553,  23.26 ,  21.218,  19.077,  57.356,  63.366,
        59.651,  54.063,  53.53 ,  53.558,  45.886,  32.178,  19.637,
        11.834,   9.169,  11.937,  19.785,  28.63 ,  32.83 ,  31.493,
        27.755,  23.937,  21.07 ,  19.588,  38.717,  52.881,  59.972,
        60.537,  59.454,  56.755,  49.02 ,  38.089,  28.157,  20.954,
        17.697,  20.163,  28.276,  37.034,  39.7  ,  35.338,  28.654,
        23.817,  21.638,  20.28 ,  21.033,  35.817,  50.437,  61.104,
        65.832,  62.404,  51.805,  44.59 ,  36.74 ,  28.7  ,  24.49 ,
        22.406,  19.109,   9.199,  19.639,  66.981,  68.789,  37.479,
        21.13 ,  15.

In [32]:
print('Are there voxel values more than 72?')
max(Un_voxel) > 72

Are there voxel values more than 72?


True

In [33]:
print('how many percentage of voxel values are more than 72?')

print(round(sum((np.array(Un_voxel) >72)*1)/len(Un_voxel),2)*100,'%')

how many percentage of voxel values are more than 72?
19.0 %


**Left Femur Head**

In [34]:
# max dose to be voxel 50
Left_voxel = []
every = 0
for i in Left_index:
    every = round(voxels[i].solution_value(),2)
    Left_voxel.append(every)
print('left femur head voxel values')
np.array(Left_voxel)

left femur head voxel values


array([29.7 , 25.4 , 40.  , 31.23, 24.14, 16.7 , 44.57, 31.13, 19.69,
       10.7 , 49.19, 29.04, 14.3 ,  5.87, 52.24, 27.83, 11.77,  4.04,
       53.45, 30.39, 14.54,  5.99, 52.68, 35.8 , 22.11, 12.07, 40.  ,
       31.06])

In [35]:
zl1 = np.array([[x.solution_value() for x in z_l_1]])
print('distances to 79.2 if voxel value is above 50\n',zl1)

distances to 79.2 if voxel value is above 50
 [[0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         2.2351535  0.         0.         0.
  3.45226567 0.         0.         0.         2.68244051 0.
  0.         0.         0.         0.        ]]


In [36]:
print('are there values more than 50?')
print(max(Left_voxel) > 50)

print('how many percentage of voxel values are more than 50?')

print(round(sum((np.array(Left_voxel) >50)*1)/len(Left_voxel),2)*100,'%')

are there values more than 50?
True
how many percentage of voxel values are more than 50?
11.0 %


In [37]:
# at most 15% of the left femur head should receive > 40
y_left_value = np.array([t.solution_value() for t in y_left])
print('voxel that are larger that 40 are indicated by 1 \n',y_left_value)

voxel that are larger that 40 are indicated by 1 
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0.
 0. 0. 0. 0.]


In [38]:
test_left_femur = np.array(list(map(lambda x: 1 if x >40.00 else 0,Left_voxel)))
print(round(sum(test_left_femur)/len(test_left_femur),4), ' of the left femur voxels are above 40')

0.1786  of the left femur voxels are above 40


In [39]:
print('voxel that are larger that 40 are indicated by 1 \n',test_left_femur)

voxel that are larger that 40 are indicated by 1 
 [0 0 0 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 0 0]


In [40]:
test_left_femur == y_left_value

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True])

In [41]:
print('the number of constraints we have to set loose to be above 40',z_y_left.solution_value())

the number of constraints we have to set loose to be above 40 1.0


**Right Femur Head**

In [42]:
# max dose to a voxel: 50 
Right_voxel = []
every = 0
for i in Right_index:
    every = round(voxels[i].solution_value(),2)
    Right_voxel.append(every)
print("right femur head voxel values")
np.array(Right_voxel)

right femur head voxel values


array([19.64, 34.91,  3.25,  8.83, 20.22, 38.95,  0.95,  3.43, 10.54,
       26.25,  0.28,  1.45,  6.11, 18.95,  0.21,  1.25,  5.67, 18.26,
        0.63,  2.55,  8.8 , 24.01,  2.19,  6.4 , 16.32, 35.79, 14.6 ,
       29.18])

In [43]:
# slack variable needed to fullfill the constraints
zrl1 = np.array([[x.solution_value() for x in z_rl_1]])
print('distances to 79.2 if voxel value is above 50\n',zrl1)

distances to 79.2 if voxel value is above 50
 [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]]


In [44]:
# check if there's any right voxel value that is bigger than 50
print('are there values more than 50?')
print(max(Right_voxel) > 50)

print('how many percentage of voxel values are more than 50?')

print(round(sum((np.array(Right_voxel) >50)*1)/len(Right_voxel),2)*100,'%')

are there values more than 50?
False
how many percentage of voxel values are more than 50?
0.0 %


In [45]:
# at most 15% of the right  femur head should receive > 40
y_right_value = np.array([t.solution_value() for t in y_right])
print('voxel that are larger that 40 are indicated by 1 \n',y_right_value)

voxel that are larger that 40 are indicated by 1 
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]


In [46]:
test_right = np.array(list(map(lambda x: 1 if x > 40.00 else 0,Right_voxel)))
print(round(sum(test_right)/len(test_right),4), ' of the right femur voxels are above 40')

0.0  of the right femur voxels are above 40


In [47]:
print('voxel that are larger that 40 are indicated by 1 \n',test_right)

voxel that are larger that 40 are indicated by 1 
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [48]:
test_right == y_right_value

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True])

In [49]:
print('the number of constraints we have to set loose to be above 40',z_y_right.solution_value())

the number of constraints we have to set loose to be above 40 0.0
